In [2]:
# import dependencies
import os
import yaml
import mysql.connector
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

## Data Ingestion
##### In this component  we will retreive/ fetch data from different data sources such as databases, data lakes data warehouses for simplicity we are fetching from mysql.

In [11]:
import mysql.connector
import pandas as pd
import os

class DataIngestion:
    def __init__(self):
        try:
            # Make the database connection
            self.conn = mysql.connector.connect(
                host='127.0.0.1',
                user='root',
                password='Bilal@123',
                database='rainfall'
            )
            # Make the cursor
            self.cursor = self.conn.cursor()
            print("Connection is Established")
        
        except Exception as e:
            print(f"Error during connection: {e}")
    
    def fetch_data(self):
        try:
            # Make the folder structure
            folder = 'data'
            os.makedirs(folder, exist_ok=True)  # Create the folder if it doesn't exist
            filepath = os.path.join(folder, 'data.csv')
            
            # Select the table
            self.cursor.execute("SELECT * FROM rainfall")
            
            # Fetch all the data
            data = self.cursor.fetchall()
            
            # Fetch all the column names
            columns = [i[0] for i in self.cursor.description]
            
            # Convert the data into a DataFrame
            data = pd.DataFrame(data, columns=columns)
            
            # Dump the data into the file path in CSV format
            data.to_csv(filepath, index=False)
            print(f"Dataset has been saved at {filepath}")
        
        except Exception as e:
            print(f"Error during data fetching: {e}")

if __name__ == "__main__":
    obj = DataIngestion()
    obj.fetch_data()


Connection is Established
Dataset has been saved at data\data.csv


## Data Validation

##### In this component we will compare the features from real time prediction with the features we had while training if there is any mismatch we will trigger it and stop the further process

In [31]:
# Read the dataset
data = pd.read_csv("data/data.csv")

# Get all the features and convert them into a list
all_cols = [col for col in data.columns if col != 'rainfall']

# Get the schema columns
with open('schema.yaml', 'r') as yaml_file:
    schema = yaml.load(yaml_file, Loader=yaml.SafeLoader)

# Get the schema columns from the dictionary and convert to a list
all_schema = list(schema['COLUMNS'].keys())

# Initialize validation status
validation_status = True  # Assume valid unless a mismatch is found

# Iterate over all_cols to validate
for col in all_cols:
    if col not in all_schema:
        print(f"{col} was not in schema")
        validation_status = False
        break  # Exit loop if any column fails validation

# Make a folder structure for validation
folder = 'data_validation'
os.makedirs(folder, exist_ok=True)

# Set the validation file path
filepath = os.path.join(folder, "validation.txt")

# Write the validation status to the file
with open(filepath, 'w') as f:
    f.write(str(validation_status))  # Convert Boolean to string

print(f"Validation status has been saved at {filepath}")


Validation status has been saved at data_validation\validation.txt


In [30]:
data.head(3)

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,rainfall,sunshine,winddirection,windspeed
0,1,1025.9,19.9,18.3,16.8,13.1,72,49,yes,9.3,80,26.3
1,2,1022.0,21.7,18.9,17.2,15.6,81,83,yes,0.6,50,15.3
2,3,1019.7,20.3,19.3,18.0,18.4,95,91,yes,0.0,40,14.2


In [ ]:
class DataValidation:
    
    def __init__(self):
        
        # read the dataset
        self.data= pd.read_csv("data/data.csv")
        